In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests

### setting up for api requests

In [77]:
# reading in tokens
with open('spotify_stuff.json', 'r') as f:
    keys = json.load(f)

# getting token
response=requests.post('https://accounts.spotify.com/api/token', data={'grant_type' : 'client_credentials'}, auth = (keys['client_id'], keys['client_secret'])) 
print(response)

# saving token for further use
token_info = json.loads(response.text)
token = 'Bearer ' + token_info['access_token']

<Response [200]>


### First, search for each track, then save track id and artists ids from top result

In [97]:
# example search request
body_params = {'q':'Havana', 'type':'track'}
head = {'Authorization':token, "Accept": "application/json", "Content-Type":"application/json"}
response = requests.get('https://api.spotify.com/v1/search', params=body_params, headers=head) 
s = json.loads(response.text)

In [98]:
# path to top result
#s['tracks']['items'][0]

In [99]:
# can save just the song id or the whole href (probably want just the id bc going to get the features)
print(s['tracks']['items'][0]['href'])
print(s['tracks']['items'][0]['id'])

https://api.spotify.com/v1/tracks/0ofbQMrRDsUaVKq2mGLEAb
0ofbQMrRDsUaVKq2mGLEAb


In [100]:
# a song can have multiple artists (need to iterate through), but can get id's from just search
s['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/4nDoRrQiYLoBzwC5BhVJzF'},
  'href': 'https://api.spotify.com/v1/artists/4nDoRrQiYLoBzwC5BhVJzF',
  'id': '4nDoRrQiYLoBzwC5BhVJzF',
  'name': 'Camila Cabello',
  'type': 'artist',
  'uri': 'spotify:artist:4nDoRrQiYLoBzwC5BhVJzF'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/50co4Is1HCEo8bhOyUWKpn'},
  'href': 'https://api.spotify.com/v1/artists/50co4Is1HCEo8bhOyUWKpn',
  'id': '50co4Is1HCEo8bhOyUWKpn',
  'name': 'Young Thug',
  'type': 'artist',
  'uri': 'spotify:artist:50co4Is1HCEo8bhOyUWKpn'}]

In [101]:
# example of features available for a track
head = {'Authorization':token, "Accept": "application/json", "Content-Type":"application/json"}
response = requests.get('https://api.spotify.com/v1/audio-features/0ofbQMrRDsUaVKq2mGLEAb', headers=head) 
json.loads(response.text)

{'acousticness': 0.186,
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0ofbQMrRDsUaVKq2mGLEAb',
 'danceability': 0.768,
 'duration_ms': 216897,
 'energy': 0.517,
 'id': '0ofbQMrRDsUaVKq2mGLEAb',
 'instrumentalness': 3.8e-05,
 'key': 7,
 'liveness': 0.104,
 'loudness': -4.323,
 'mode': 0,
 'speechiness': 0.0312,
 'tempo': 104.992,
 'time_signature': 4,
 'track_href': 'https://api.spotify.com/v1/tracks/0ofbQMrRDsUaVKq2mGLEAb',
 'type': 'audio_features',
 'uri': 'spotify:track:0ofbQMrRDsUaVKq2mGLEAb',
 'valence': 0.418}

In [102]:
# example of getting an artist from an id (['genres'] is what we want)
head = {'Authorization':token, "Accept": "application/json", "Content-Type":"application/json"}
response = requests.get('https://api.spotify.com/v1/artists/50co4Is1HCEo8bhOyUWKpn', headers=head) 
json.loads(response.text)

{'external_urls': {'spotify': 'https://open.spotify.com/artist/50co4Is1HCEo8bhOyUWKpn'},
 'followers': {'href': None, 'total': 2081764},
 'genres': ['dwn trap', 'gangster rap', 'pop rap', 'rap', 'trap music'],
 'href': 'https://api.spotify.com/v1/artists/50co4Is1HCEo8bhOyUWKpn',
 'id': '50co4Is1HCEo8bhOyUWKpn',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/1eb847e76b0c8391cd9ec6dff07e93b86ec33d77',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/64713d8ad9d6f05dc5f55268f8fd1e7e2e89888f',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/25cdb6baf095e1e1f094a256bb6b3b1eceacc143',
   'width': 160}],
 'name': 'Young Thug',
 'popularity': 94,
 'type': 'artist',
 'uri': 'spotify:artist:50co4Is1HCEo8bhOyUWKpn'}

### reading in radio data to build a lookup table

In [ ]:
data = pd.DataFrame(columns=['artist', 'time', 'title', 'day', 'month', 'station', 'start_time', 'end_time'])

for file in list(map(lambda x: 'data/' + x, ['94.9_900_songs.csv', 'second_94.9.csv', 'kmel_songs.csv', 'star_songs.csv', 'not_even_in_our_timezone.csv'])):
    df = pd.read_csv(file)
    df = df[::-1].reset_index(drop=True)
    df['start_time'] = pd.to_datetime('2017' + (df.month*100+df.day).apply(str) + ' ' + df.time,format='%Y%m%d %I:%M %p')
    dt_2 = df['start_time'][1:].reset_index(drop=True)
    dt_2.set_value(value=df['start_time'].iloc[-1] + pd.Timedelta(minutes=3), label=len(dt_2))
    df['end_time'] = dt_2
    data = pd.concat([data, df])

print('Rows:', len(data))
data.head()